Import libraries

In [1]:
import pandas as pd
import os
import numpy as np

First load the excel sheet with three sheets which are:

*   Parameters table
*   Data table
*   Population projection

In [13]:
os.chdir(r'C:\Users\mkrzakala\OneDrive - IBS\BENEFIT PROJEKT\analiza\simulation')
parameters_df = pd.read_excel('Input.xlsx', sheet_name='parameters')
data_df = pd.read_excel('Input.xlsx', sheet_name='data_2024')

print(parameters_df.head())
print(data_df.head())
df = data_df

     variable  s_n_men  w_s_men  s_n_women  w_s_women  age_1  age_2  age_3  \
0  employment    1.038        1      1.052          1  25-29  30-34  35-39   
1     absence    0.790        1      0.790          1  20-24  25-29  30-34   
2      cancer    0.990        1      0.990          1  15-19  20-24  25-29   
3   mortality    0.900        1      0.820          1  20-24  25-29  30-34   

   age_4  age_5  age_6  age_7  age_8  age_9 age_10 age_11 age_12  age_13  \
0  40-44    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN   
1  35-39  40-44  45-49  50-54  55-59  60-64  65-69  70-74  75-79  80-100   
2  30-34  35-39  40-44  45-49  50-54  55-59  60-64  65-69  70-74   75-79   
3  35-39  40-44  45-49  50-54  55-59  60-64  65-69  70-74  75-79  80-100   

   age_14  
0     NaN  
1     NaN  
2  80-100  
3     NaN  
   year    age sex        s1        s2        s3    population  mortality  \
0  2024  15-19   K  0.400000  0.363636  0.236364  1.131077e+06   0.000254   
1  2024  15-19

Now calculate the baseline values for each variable

In [14]:
df["absence_pc"] = df["absence"]/df["working"]

In [15]:
age_groups_20_64 = [
    "20-24", "25-29", "30-34", "35-39", "40-44",
    "45-49", "50-54", "55-59", "60-64"
]

# Create new column with population where age group is in 20–64 range, otherwise NaN
df["population_20_64"] = df.apply(
    lambda row: row["population"] if row["age"] in age_groups_20_64 else 0,
    axis=1
)

prepare parameter values

In [16]:
for _, row in parameters_df.iterrows():
    print(row['variable'])

employment
absence
cancer
mortality


In [17]:
# First, get a list of age column names from df_params
age_cols = [col for col in parameters_df.columns if col.startswith("age_")]

# For each variable (row) in df_params
for _, row in parameters_df.iterrows():
    variable = row["variable"]
    valid_ages = row[age_cols].dropna().tolist()  # list of age groups for this variable

    # Prepare output column names
    col_s_n = f"{variable}_s_n"
    col_w_s = f"{variable}_w_s"

    # Initialize with default value 1
    df[col_s_n] = 1.0
    df[col_w_s] = 1.0

    # Apply s_n value (static) based on age and sex
    df.loc[(df["sex"] == "M") & (df["age"].isin(valid_ages)), col_s_n] = row["s_n_men"]
    df.loc[(df["sex"] == "K") & (df["age"].isin(valid_ages)), col_s_n] = row["s_n_women"]

    # Apply w_s value (dynamic) based on age and sex
    df.loc[(df["sex"] == "M") & (df["age"].isin(valid_ages)), col_w_s] = row["w_s_men"]
    df.loc[(df["sex"] == "K") & (df["age"].isin(valid_ages)), col_w_s] = row["w_s_women"]


Assign Shock that 50% of inactive goes to active (half to s2 and half to s3)

In [33]:
df['s1_as'] = df['s1'] - 0.1
df['s2_as'] = df['s2'] + 0.1 
df['s3_as'] = df['s3']

In [35]:
list = ["mortality", "employment", "absence", "cancer"]


for idx, i in enumerate(list):
    name = i+"_s1"

    df[i+"_denominator"] = df.s1 + df.s2 * df[i + "_s_n"] + df.s3 * df[i + "_s_n"] * df[i + "_w_s"]
    df[i+"_s1"] = df[i] / df[i+"_denominator"]
    df[i+"_s2"] = df[i+"_s1"] * df[i + "_s_n"]
    df[i+"_s3"] = df[i+"_s2"] * df[i + "_w_s"]
    
    df[i+"_bs"] = df['s1'] * df[i+"_s1"] + df['s2'] * df[i+"_s2"] + df['s3'] * df[i+"_s3"]
    df[i+"_as"] = df['s1_as'] * df[i+"_s1"] + df['s2_as'] * df[i+"_s2"] + df['s3_as'] * df[i+"_s3"]

Now calculate the theoretical values of all variables based on given parameters

Now we want to calculate the averages/sums for the population

For Mortality weighted average by demographic group size, for employment rate weighted average by number of working people, for absence simple sum, for cancer average by population

In [36]:
data = {
    "name": ["mortality", "employment", "absence", "cancer"],
    "type": ["average", "average", "sum", "average"],
    "weight": ["population", "population_20_64", "na", "population"]
}

# Create the DataFrame
results = pd.DataFrame(data)

In [38]:
for idx, row in results.iterrows():
    name = row["name"]
    agg_type = row["type"]
    weight_key = row["weight"]

    if agg_type == "sum":
        result_bs = sum(df[name + '_bs'])
        result_as = sum(df[name + '_as'])
    elif agg_type == "average":
        if weight_key != "na":
            w = df[weight_key]
            v_bs = df[name + '_bs']
            v_as = df[name + '_as']
            result_bs = np.average(v_bs, weights=w)
            result_as = np.average(v_as, weights=w)

        else:
            result = "n/a"
    else:
        result = "n/a"

    results.at[idx, "result_bs"] = result_bs
    results.at[idx, "result_as"] = result_as

results.to_excel("output.xlsx", index=False)